# Imports

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import data_loaders, prepare_submission, net

import matplotlib.pyplot as plt
import numpy as np
import torch                     
from torch.autograd import Variable

# Data

In [6]:
train_raw, train_keys_raw = data_loaders.training_images()
"""
test, test_keys = data_loaders.test_images()
masks = data_loaders.training_masks()
s_masks = data_loaders.single_masks(masks)
"""

'\ntest, test_keys = data_loaders.test_images()\nmasks = data_loaders.training_masks()\ns_masks = data_loaders.single_masks(masks)\n'

In [7]:
# For now, let us reduce our input data to just things that are 256x256
train, train_keys = [], []%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
for i in range(len(train_raw)):
    if train_raw[i].shape == (256, 256):
        train.append(train_raw[i])
        train_keys.append(train_keys_raw[i])
print(len(train), len(train_keys))

334 334


# UNET

In [8]:
n = net.u_net()

In [9]:
# Run something through it as a sanity check
inp = Variable(torch.randn(1, 1, 572, 572))
n(inp).shape

torch.Size([1, 1, 388, 388])